In [8]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math
print(f"Đã import các thư viện cần thiết")

Đã import các thư viện cần thiết


In [9]:
class SimpleCNN(nn.Module):
    #hàm khởi tạo: Định nghĩa các lớp sẽ sử dụng
    def __init__(self,num_classes=1,img_size=150):
        super().__init__() #gọi hàm khởi tạo của lớp cha
        print(f"Khởi tạo SimpleCNN với num_classes={num_classes},img_size={img_size}")

        #--phần trích xuất đặc trưng(Convulation Base)--
        self.features=nn.Sequential(
            #Block 1
            #Lớp tích chập đầu tiên
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=1), #Lớp tích chập đầu tiên
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),

            #Block 2
            #Lớp tích chập thứ hai
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),

            #Block 3
            #Lớp tích chập thứ ba
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        
        #--Phần phân loại(Classifier)--
        with torch.no_grad():
            #tạo 1 dummy input đúng với kích thước ban đầu
            dummy_input=torch.zeros(1,3,img_size,img_size)
            #cho dummy input đi qua phần trích xuất đặc trưng
            output_features=self.features(dummy_input)
            #lấy số lượng phần tử của output(trừ batch)
            self.flatten_size=output_features.numel() #numel để tính tổng phần tử
            print(f"Flatten size: {self.flatten_size}")
            if self.flatten_size==0:
                raise ValueError("Flatten size is 0. Check your model architecture.")
        
        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_size,512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512,num_classes),
            nn.Sigmoid()
        )

    #hàm định nghĩa cách dữ liệu đi qua các lớp đã định nghĩa
    def forward(self,x):
        #trích xuất đặc trưng
        x=self.features(x)
        #phân loại
        x=self.classifier(x)
        return x


# Phần kiểm tra

In [10]:
if __name__=='__main__':
    print(f"Chạy model.ipynb để kiểm tra")
    test_img_size=150
    test_num_classes=1
    #tạo model thử nghiệm 
    test_model=SimpleCNN(num_classes=test_num_classes,img_size=test_img_size)
    print(f"\n Kiến trúc model: ")
    print(test_model)

    #tạo 1 batch dữ liệu để kiểm tra forward
    dummy_input=torch.randn(4,3,test_img_size,test_img_size) #1 ảnh RGB
    print(f"\n Kích thước đầu vào: {dummy_input.shape}")

    #cho dữ liệu đi qua model
    try:
        output=test_model(dummy_input)
        print(f"Forrward pass thành công")
        print(f"Kích thước đầu ra: {output.shape}")
    except Exception as e:
        print(f"Forward pass thất bại: {e}")
    
    #kiểm tra bằng torch summary
    try:
        from torchsummary import summary
        print("\n--- Tóm tắt Model bằng torchsummary ---")
        # Cần cung cấp kích thước input (C, H, W) và device
        summary(test_model, input_size=(3, test_img_size, test_img_size), device="cpu")
    except ImportError:
        print("torchsummary chưa được cài đặt. Bỏ qua tóm tắt chi tiết.")
    except Exception as e:
         print(f"Lỗi khi dùng torchsummary: {e}")



Chạy model.ipynb để kiểm tra
Khởi tạo SimpleCNN với num_classes=1,img_size=150
Flatten size: 20736

 Kiến trúc model: 
SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=20736, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

 Kích thước đầu vào: torch.Size([4, 3, 15